# Testing Notebook 02:

In [1]:
from typing import List
import pandas as pd 
import numpy as np
import plotly.express as px

from statdepth.depth.depth import FunctionalDepth

ImportError: cannot import name 'FunctionalDepth' from 'statdepth.depth.abstract' (/Users/julian/opt/miniconda3/envs/base-data-science/lib/python3.8/site-packages/statdepth/depth/abstract.py)

In [ ]:
!where python

Generate a toy DataFrame for testing

In [ ]:
df = pd.DataFrame(np.random.rand(5, 10), columns=list('ZXCVBNMLKJ'))
px.line(df)


In [4]:
df = pd.DataFrame(np.random.rand(5, 10))

bd = BandDepth([df], J=3, relax=True)
bd

0    0.472778
1    0.781111
2    0.411111
3    0.513889
4    0.842778
5    0.575556
6    0.760556
7    0.555000
8    0.698889
9    0.555000
dtype: float64

In [5]:
bd.ordered()

4    0.842778
1    0.781111
6    0.760556
8    0.698889
5    0.575556
9    0.555000
7    0.555000
3    0.513889
0    0.472778
2    0.411111
dtype: float64

In [6]:
bd.median()

4    0.842778
dtype: float64

In [6]:
from typing import get_type_hints, Dict, Union, List

def dummy(l: int, s: str, j: Union[List[str], int]) -> bool:
    return False

hints = get_type_hints(dummy)


In [7]:
hints

{'l': int, 's': str, 'j': typing.Union[typing.List[str], int], 'return': bool}

In [8]:
pd.DataFrame(df)

,0,1,2,3,4,5,6,7,8,9
0,0.645003,0.997211,0.894304,0.352930,0.151210,0.871791,0.252705,0.724823,0.504422,0.603860
1,0.147823,0.433712,0.114693,0.905751,0.131263,0.916580,0.162851,0.730408,0.961714,0.083790
2,0.527697,0.017230,0.998144,0.811273,0.848748,0.213646,0.414577,0.585114,0.760536,0.706195
3,0.667622,0.257833,0.263420,0.739945,0.140056,0.114974,0.460889,0.110615,0.029332,0.806295
4,0.371355,0.322612,0.031154,0.161339,0.134129,0.639753,0.737802,0.335094,0.332832,0.177039
